In [ ]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
import os

# try:
#     device_name = os.environ['COLAB_TPU_ADDR']
#     TPU_ADDRESS = 'grpc://' + device_name
#     print('Found TPU')
# except KeyError:
#     print('TPU not found')

# tf.config.experimental_connect_to_cluster(TPU_ADDRESS)
# resolver = tf.distribute.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)
# tf.tpu.experimental.initialize_tpu_system(resolver)
# strategy = tf.distribute.experimental.TPUStrategy(resolver) 

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu="")
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices("TPU"))

strategy = tf.distribute.TPUStrategy(resolver)



INFO:tensorflow:Initializing the TPU system: grpc://10.109.72.50:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.109.72.50:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
import numpy as np
# Import glove embeddings
embeddings_index = {}
embeddingFile = open("drive/MyDrive/glove.6B.100d.txt", encoding="utf-8")
for line in embeddingFile:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype="float32")
    embeddings_index[word] = coefs
embeddingFile.close()
print("Number of words in glove embeddings = ", len(embeddings_index))

Number of words in glove embeddings =  400000


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer


def pad(x, length=None):
    if length is None:
        length = max([len(sentence) for sentence in x])
    return pad_sequences(x, maxlen=length, padding="post")


def tokenize(sentences):
    tokenizer = Tokenizer(oov_token="")
    tokenizer.fit_on_texts(sentences)
    return tokenizer.texts_to_sequences(sentences), tokenizer

def load_data(path):
    """
    Load dataset
    """
    input_file = os.path.join(path)
    with open(input_file, "r") as f:
        data = f.read()
 
    return data.split('\n')

normal_sentences = load_data("drive/MyDrive/normal.training.txt")
simple_sentences = load_data("drive/MyDrive/simple.training.txt")

sentence_amount = len(normal_sentences)
normal_sentences = normal_sentences[:sentence_amount]
simple_sentences = simple_sentences[:sentence_amount]
 
print("Normal Sentences Loaded: ", len(normal_sentences))
print("Simple Sentences Loaded: ", len(simple_sentences))
 
wikipedia_sentences = []
simple_wikipedia_sentences = []
 
for sentence in normal_sentences:
    #sentence = re.sub("[^\\w\\s]", "", sentence)
    sentence_split = sentence.split(" ")
 
    while '' in sentence_split:
        sentence_split.remove('')
 
    wikipedia_sentences.append(sentence_split)
 
for sentence in simple_sentences:
    #sentence = re.sub("[^\\w\\s]", "", sentence)
    sentence_split = sentence.split(" ")
 
    while '' in sentence_split:
        sentence_split.remove('')
 
    simple_wikipedia_sentences.append(sentence_split)
 
 
indexesToRemove = []
 
# Remove "weird" sentences
for idx, sentence in enumerate(wikipedia_sentences):
    if len(sentence) > 75 or len(simple_wikipedia_sentences[idx]) > 75:
        indexesToRemove.append(idx)
 
wikipedia_sentences = [v for i,v in enumerate(wikipedia_sentences) if i not in indexesToRemove]
simple_wikipedia_sentences = [v for i,v in enumerate(simple_wikipedia_sentences) if i not in indexesToRemove]
 
 
# Tokenize input/output sentences
input_sequences, input_tokenizer = tokenize(wikipedia_sentences)
print("First Input: ", input_sequences[0])
output_sequences, output_tokenizer = tokenize(simple_wikipedia_sentences)
print("First output: ", output_sequences[0])
 
# Get vocab size of simple and regular wikipedia
regular_vocab_size = len(input_tokenizer.word_index)
simple_vocab_size = len(output_tokenizer.word_index)
 
# Pad input/output sentences
input_sequences = pad(input_sequences)
output_sequences = pad(output_sequences)
 
# Sparse categorical_crossentropy function requires the labels to be in 3 dimensions
output_sequences = output_sequences.reshape(*output_sequences.shape, 1)
 
# Get maximum length of simple and regular wikipedia sentences
max_sentence_length = input_sequences.shape[1]
max_simple_sentence_length = output_sequences.shape[1]
 
# Add additional padding so they're the same length
if(input_sequences.shape[1] > output_sequences.shape[1]):
    output_sequences = pad(output_sequences, input_sequences.shape[1])
else:
    input_sequences = pad(input_sequences, output_sequences.shape[1])
 
 
print("Max Regular wikipedia sentence length:", max_sentence_length)
print("Max Simple wikipedia sentence length:", max_simple_sentence_length)
print("Regular wikipedia vocab size:", regular_vocab_size)
print("Simple wikipedia vocab size:", simple_vocab_size)
 
input_sequences = input_sequences.reshape((-1, output_sequences.shape[-2], 1))
 

Normal Sentences Loaded:  123627
Simple Sentences Loaded:  123627
First Input:  [62, 555, 13627, 28, 54086, 6, 8, 27182, 157, 3, 2507, 354, 88, 12, 2, 7545, 95, 39345, 11207, 4]
First output:  [2576, 11, 2, 8634, 4245, 13, 2, 574, 839, 7098, 92, 2, 736, 1099, 170, 9, 2, 711, 35151, 3, 54, 58, 167, 4245, 8014, 575, 11, 3900, 6, 7, 35151, 155, 4, 2576, 82, 2, 9345, 3]
Max Regular wikipedia sentence length: 75
Max Simple wikipedia sentence length: 75
Regular wikipedia vocab size: 110223
Simple wikipedia vocab size: 100602


In [ ]:
embedding_matrix = np.zeros((regular_vocab_size, 100))


In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay

early_stopping = EarlyStopping(monitor='loss', patience=3)
checkpoint = ModelCheckpoint("drive/MyDrive/tpuModel5.hdf5", monitor='loss', verbose=1, save_best_only=True, mode='auto', period=10)

lr_schedule = ExponentialDecay(
    1e-3,
    decay_steps = 1e5,
    decay_rate = 0.96,
    staircase=True
)

with strategy.scope():
  network = Sequential()
  network.add(Embedding(regular_vocab_size +1, 100, input_length=input_sequences.shape[1]))
  network.add(Bidirectional(GRU(300, return_sequences=False)))
  network.add(RepeatVector(max_sentence_length))
  network.add(Bidirectional(GRU(300, return_sequences=True)))
  network.add(TimeDistributed(Dense(simple_vocab_size + 1, activation="softmax")))

  network = Sequential()
  network.add(Embedding(regular_vocab_size +1, 100, input_length=input_sequences.shape[1]))
  network.add(Dropout(0.4))
  network.add(Bidirectional(GRU(300, return_sequences=False)))
  network.add(RepeatVector(max_sentence_length))
  network.add(Bidirectional(GRU(300, return_sequences=True)))
  network.add(Dropout(0.4))
  network.add(TimeDistributed(Dense(simple_vocab_size + 1, activation="softmax")))
  
  network.compile(loss="sparse_categorical_crossentropy", optimizer=Adam(lr_schedule), metrics=["accuracy"])
  network.fit(input_sequences, output_sequences, batch_size=512, epochs=300, validation_split=0.2, callbacks=[checkpoint, early_stopping])

#   network.load_weights("drive/MyDrive/tpuModel2.hdf5")
#   network.compile(loss="sparse_categorical_crossentropy", optimizer=Adam(1e-4), metrics=["accuracy"])
#   network.fit(input_sequences, output_sequences, batch_size=1024, epochs=2000, validation_split=0.2, callbacks=[checkpoint])

#   network.load_weights("drive/MyDrive/tpuModel2.hdf5")
#   network.compile(loss="sparse_categorical_crossentropy", optimizer=Adam(1e-5), metrics=["accuracy"])
#   network.fit(input_sequences, output_sequences, batch_size=1024, epochs=2000, validation_split=0.2, callbacks=[checkpoint])

#   network.load_weights("drive/MyDrive/tpuModel2.hdf5")
#   network.compile(loss="sparse_categorical_crossentropy", optimizer=Adam(1e-6), metrics=["accuracy"])
#   network.fit(input_sequences, output_sequences, batch_size=1024, epochs=1000, validation_split=0.2, callbacks=[checkpoint])

#   network.load_weights("drive/MyDrive/tpuModel2.hdf5")
#   network.compile(loss="sparse_categorical_crossentropy", optimizer=Adam(1e-7), metrics=["accuracy"])
#   network.fit(input_sequences, output_sequences, batch_size=1024, epochs=1000, validation_split=0.2, callbacks=[checkpoint])



  


# network.save_weights("drive/MyDrive/network2.h5")







In [ ]:
def outputToText(logits, tokenizer):
  index_to_words = {id: word for word, id in tokenizer.word_index.items()}
  index_to_words[0] = ''
  predictions = [index_to_words[prediction] for prediction in np.argmax(logits, 1)]

  noDuplicatePredictions = []

  for idx, prediction in enumerate(predictions):
    if idx == 0:
      noDuplicatePredictions.append(prediction)
    else:
      if predictions[idx - 1] != prediction:
        noDuplicatePredictions.append(prediction)
  return " ".join(noDuplicatePredictions)



for i in range(1, 5):
  target_sequence = input_sequences[i].reshape(1, 75, 1)
  prediction = network.predict(target_sequence)
  output = outputToText(prediction[0], output_tokenizer)
  print("Input:", wikipedia_sentences[i])
  print("Expected:", simple_wikipedia_sentences[i])
  print("Result:", output)


Input: ['Royal', 'Rumble', 'was', 'the', 'twentieth', 'annual', 'Royal', 'Rumble', 'professional', 'wrestling', 'pay-per-view', 'event', 'produced', 'by', 'World', 'Wrestling', 'Entertainment', '.']
Expected: ['Royal', 'Rumble', 'was', 'the', 'twentieth', 'yearly', 'Royal', 'Rumble', 'professional', 'wrestling', 'pay-per-view', 'event', 'made', 'by', 'World', 'Wrestling', 'Entertainment', '.']
Result: the rumble wrestling pay-per-view the wrestling . 
Input: ['Janko', 'Prunk', 'is', 'a', 'Slovenian', 'historian', 'of', 'modern', 'history', '.']
Expected: ['Janko', 'Prunk', ',', 'is', 'a', 'Slovenian', 'historian', 'of', 'modern', 'history', '.']
Result: janko prunk is a slovenian historian of history . 
Input: ['Sometimes', 'the', 'balalaika', 'is', 'tuned', '``', 'guitar', 'style', "''", 'to', 'G-B-D', ',', 'making', 'it', 'easier', 'to', 'play', 'for', 'Russian', 'guitar', 'players', ',', 'although', 'balalaika', 'purists', 'frown', 'on', 'this', 'tuning', '.']
Expected: ['The', 'Bal

In [ ]:
import pickle

with open('drive/MyDrive/input_tokenizer5.pickle', 'wb') as handle:
    pickle.dump(input_tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('drive/MyDrive/output_tokenizer5.pickle', 'wb') as handle:
    pickle.dump(output_tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

model_json = network.to_json()
with open("drive/MyDrive/network5.json", "w") as json_file:
    json_file.write(model_json)
    json_file.close()

In [ ]:
text = "Royal Rumble was the twentieth annual Royal Rumble professional wrestling pay-per-view event produced by World Wrestling"

sequence = pad(input_tokenizer.texts_to_sequences([text]), 75)[0]
sequence = sequence.reshape(1, 75, 1)
#print(sequence)

prediction = network.predict(sequence)
output = outputToText(prediction[0], output_tokenizer)
print(output)



the rumble the . 
